In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from utils import *
from utils import grid_search_sarima


ImportError: cannot import name 'grid_search_sarima' from 'utils' (w:\repo\Price_prediction\utils.py)

In [25]:
from utils import plot_acf_pacf


ImportError: cannot import name 'plot_acf_pacf' from 'utils' (w:\repo\Price_prediction\utils.py)

In [13]:
df = pd.read_csv("bigqueryDatabases/prices.csv")
df = filtrar_ub(df)

In [14]:
df

year_month
2007-01-01     9.198165
2007-02-01     9.286962
2007-03-01     9.051190
2007-04-01     9.273183
2007-05-01     9.060988
                ...    
2025-01-01    14.121816
2025-02-01    13.820212
2025-03-01    13.902885
2025-04-01    14.183055
2025-05-01    13.517076
Name: price, Length: 221, dtype: float64

In [15]:
#ya sabemos que la seri es estacionaria

df_1 = filtrar_trimestre(df, trimestre=1)
df_2 = filtrar_trimestre(df, trimestre=2)
df_3 = filtrar_trimestre(df, trimestre=3)
df_4 = filtrar_trimestre(df, trimestre=4)

In [16]:

def probar_estacionariedad(df_trim, nombre=""):
    resultado = adfuller(df_trim.dropna())
    print(f"--- {nombre} ---")
    print(f"ADF Statistic : {resultado[0]:.4f}")
    print(f"p-value       : {resultado[1]:.4f}")
    print(f"Lags Used     : {resultado[2]}")
    print(f"N Obs         : {resultado[3]}")
    for key, value in resultado[4].items():
        print(f"Critical Value {key} : {value:.4f}")
    if resultado[1] < 0.05:
        print("=> La serie ES estacionaria (rechazamos H0)\n")
    else:
        print("=> La serie NO es estacionaria (no rechazamos H0)\n")

# Usar con tus dataframes trimestrales
probar_estacionariedad(df_1, "Trimestre 1")
probar_estacionariedad(df_2, "Trimestre 2")
probar_estacionariedad(df_3, "Trimestre 3")
probar_estacionariedad(df_4, "Trimestre 4")


--- Trimestre 1 ---
ADF Statistic : -1.7647
p-value       : 0.3981
Lags Used     : 0
N Obs         : 56
Critical Value 1% : -3.5529
Critical Value 5% : -2.9147
Critical Value 10% : -2.5951
=> La serie NO es estacionaria (no rechazamos H0)

--- Trimestre 2 ---
ADF Statistic : -2.8868
p-value       : 0.0469
Lags Used     : 1
N Obs         : 54
Critical Value 1% : -3.5577
Critical Value 5% : -2.9168
Critical Value 10% : -2.5962
=> La serie ES estacionaria (rechazamos H0)

--- Trimestre 3 ---
ADF Statistic : -2.9536
p-value       : 0.0395
Lags Used     : 0
N Obs         : 53
Critical Value 1% : -3.5602
Critical Value 5% : -2.9179
Critical Value 10% : -2.5968
=> La serie ES estacionaria (rechazamos H0)

--- Trimestre 4 ---
ADF Statistic : -2.6573
p-value       : 0.0817
Lags Used     : 1
N Obs         : 52
Critical Value 1% : -3.5629
Critical Value 5% : -2.9190
Critical Value 10% : -2.5974
=> La serie NO es estacionaria (no rechazamos H0)



In [17]:
# como df1 y df4 son no estacionarios, vamos a diferenciarlos
# Aplicar diferenciación de primer orden
df_1 = df_1.diff().dropna()
df_4 = df_4.diff().dropna()
# Ahora todas son estacionarias


In [ ]:
# Buscar los hiperparámetros óptimos para cada trimestre
resultados_sarima = {}

for i, df_trim in enumerate([df_1, df_2, df_3, df_4], start=1):
    print(f"\n🔍 Buscando mejores hiperparámetros para Trimestre {i}...")
    
    result = grid_search_sarima(
        series=df_trim,
        p_range=range(0, 3),
        d_range=range(0, 2),
        q_range=range(0, 3),
        P_range=range(0, 2),
        D_range=range(0, 2),
        Q_range=range(0, 2),
        s=4,  # Trimestralidad
        verbose=True  # Cambia a False si no quieres ver cada intento
    )
    
    resultados_sarima[f'Trimestre {i}'] = result


NameError: name 'df_1' is not defined

In [ ]:
print("\n hiperparámetros por trimestre:")
for trimestre, res in resultados_sarima.items():
    print(f"\n📦 {trimestre}")
    print("  Order:", res["order"])
    print("  Seasonal:", res["seasonal_order"])
    print("  AIC:", res["aic"])